In [ ]:
# 🧪 Python Kodēšana 2 – Tests (1.–4. nodarbība) ar taimeri
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from datetime import datetime
import requests
import asyncio

# Jautājumi
questions_data = [
    ("Ko dara pygame.init()?", [
        "Inicializē visus importētos pygame moduļus",
        "Palaiž spēles ciklu",
        "Ielādē spēles logu",
        "Izveido galveno varoņa sprite objektu"
    ], 0),
    # ... (pievieno atlikušos jautājumus)
]

random.shuffle(questions_data)
results = [None] * len(questions_data)
attempts = [0] * len(questions_data)
question_widgets = []
submit_button = widgets.Button(description="Iesniegt rezultātu", button_style='success')
download_button = widgets.Button(description="⬇️ Lejupielādēt rezultātu", button_style='info')
send_button = widgets.Button(description="📤 Sūtīt uz Google Formu", button_style='primary')
name_input = widgets.Text(description='Vārds:', placeholder='Ievadi savu vārdu')

form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdTSrnEP9GQajJRRCKqQMbwWGS_TL3o3B4T_fequkWmUZQCCg/formResponse"
entry_ids = {
    "Students": "entry.1649720525",
    "Datums": "entry.667959859",
    "Rezultats": "entry.940605140"
}  # un pievieno atlikumu, ja vajag

# Taimeris
time_left = 40 * 60
timer_label = widgets.Label()

async def update_timer():
    global time_left
    while time_left > 0:
        mins, secs = divmod(time_left, 60)
        timer_label.value = f"Atlikusais laiks: {mins:02d}:{secs:02d}"
        await asyncio.sleep(1)
        time_left -= 1
    timer_label.value = "⏰ Laiks beidzies!"
    submit_button.disabled = True
    for w in question_widgets:
        for child in w.children:
            if hasattr(child, 'disabled'):
                child.disabled = True
    send_to_google_form(None)

def create_question(index, question, options, correct_index):
    out = widgets.VBox()
    paired = list(enumerate(options))
    random.shuffle(paired)
    shuffled_indexes, shuffled_options = zip(*paired)
    correct_shuffled_index = shuffled_indexes.index(correct_index)

    radio = widgets.RadioButtons(options=shuffled_options)
    button = widgets.Button(description='Pārbaudīt')
    feedback = widgets.HTML()

    def on_click(b):
        attempts[index] += 1
        if radio.index == correct_shuffled_index:
            penalty = 5 * (attempts[index] - 1)
            score = max(12.5 - penalty, 0)
            results[index] = score
            feedback.value = "<span style='color:green'>✔️ Pareizi!</span>"
            radio.disabled = True
            button.disabled = True
            if all(r is not None for r in results):
                display(widgets.VBox([name_input, submit_button]))
        else:
            feedback.value = "<span style='color:red'>❌ Nepareizi. Mēģini vēlreiz.</span>"

    button.on_click(on_click)
    out.children = [widgets.HTML(f"<b>{index+1}. {question}</b>"), radio, button, feedback]
    return out

def calculate_score(_):
    clear_output()
    for w in question_widgets:
        display(w)
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    color = 'green' if final_score >= 70 else 'red'
    status = "✅ Nokārtots!" if final_score >= 70 else "❌ Nenokārtots."
    display(widgets.HTML(f"<h3>Rezultāts: <span style='color:{color}'>{final_score}%</span> – {status}</h3>"))
    display(widgets.VBox([download_button, send_button]))

def download_result(_):
    student_name = name_input.value.strip() or "Nezināms"
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    filename = f"rezultats_{student_name.replace(' ', '_')}.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"Students: {student_name}\nDatums: {now}\nRezultāts: {final_score}%\n\n")
        for i, (q, _, _) in enumerate(questions_data):
            f.write(f"{i+1}. {q} (Mēģinājumi: {attempts[i]}) – {results[i]} punkti\n")
    try:
        from google.colab import files
        files.download(filename)
    except:
        print(f"Fails saglabāts kā {filename}")

def send_to_google_form(_):
    student_name = name_input.value.strip() or "Nezināms"
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    form_data = {
        entry_ids["Students"]: student_name,
        entry_ids["Datums"]: now,
        entry_ids["Rezultats"]: f"{final_score}%"
    }
    try:
        response = requests.post(form_url, data=form_data)
        if response.status_code == 200:
            display(widgets.HTML("<h4 style='color:green;'>Rezultāti veiksmīgi nosūtīti uz Google Formu!</h4>"))
        else:
            display(widgets.HTML(f"<h4 style='color:red;'>Kļūda sūtot datus (statuss: {response.status_code})</h4>"))
    except Exception as e:
        display(widgets.HTML(f"<h4 style='color:red;'>Kļūda sūtot datus: {e}</h4>"))

submit_button.on_click(calculate_score)
download_button.on_click(download_result)
send_button.on_click(send_to_google_form)

# Rāda jautājumus
display(timer_label)
for i, (q, opts, correct) in enumerate(questions_data):
    qw = create_question(i, q, opts, correct)
    question_widgets.append(qw)
    display(qw)

asyncio.create_task(update_timer())
